In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations

from fim_doe import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt

In [2]:
from fixed_bed_model_indexed_dynamic import *

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:228)


In [3]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Argument:
        timepoints: the timesteps
        time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    add_variables(doe, timesteps=timepoints, start=time_start)
    
    add_equations(doe)
    
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68, collo=False):
    '''
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    here it's 68
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
    
    
def initialize(doe, init_point):
    '''
    Argument: 
        init_point: initial point, csv file
    '''
    # Initialize 
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    initial_bed_csv(doe, store)
    fix_initial_bed(doe)
    
def generate_exp(feed, bath, y, const_q):
    initialize_q = {}
    for t in exp_time_:
        initialize_q[t] = const_q
    print(exp_time_[0])
    dv_dict_overall = {'temp_feed': {0: feed}, 'temp_bath': {0:bath}, 'yfeed': {0: y}, 'Q': initialize_q}
    return dv_dict_overall

In [4]:
NFEt = 68
time_scale = 3200
collo = False
exp_time_ = time_points(NFEt, time_scale)

scena_all = {'scena-name': [0], 'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

initialize_point = '2021oct8_f293_b293_y15_5e6.csv'

In [5]:
def model_integrate(scena, args_list, exp_time=exp_time_, init_point=initialize_point):
    test = create_model(scena, temp_feed=293.15, temp_bath=293.15, y=0.15, doe_model=args_list[0], k_aug=args_list[1], opt=False, diff=0)
    add_model(test, exp_time)
    discretize(test)
    initialize(test, init_point)
    return test


In [6]:
# choose from model and k_aug 
model_opt = 'doe'

if (model_opt == 'doe'):
    args_ = [True, False]
    
elif (model_opt =='kaug'):
    args_ = [True, True]
    

parameter_dict = {'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#dv_pass = {'temp_feed': [0], 'temp_bath':[0], 'yfeed': [0]}
#dv_pass = {'temp_feed': None, 'temp_bath':None , 'yfeed': None}
dv_pass = {'temp_feed': None, 'yfeed': None, 'Q':exp_time_}

#measurement = ['FCO2', 'temp']
#measurement_extra = {'FCO2': [19], 'temp': [10, 19]}

t_measure = exp_time_
#measurement_pass = {'FCO2': t_measure, 'temp': t_measure}
measurement_pass = {'FCO2': {19: t_measure}, 'temp': {10: t_measure, 19:t_measure}}

In [7]:
prior_all = np.zeros((2,2))

prior_pass=np.asarray(prior_all)


In [8]:
# Define experiments
exp1 = generate_exp(313.15, 293.15, 0.15, 0)
    
sensi_opt = 'sequential_finite'

0


In [10]:
doe_object1 = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement_pass, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None,  args=args_)
result = doe_object1.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                                   scale_nominal_param_value = True, 
                                    store_output = 'dont', read_output='./2021oct_heatmap/20211028_heatmap_293_15',
                                     formula='central')

Sensitivity information is scaled by its corresponding parameter nominal value.


In [12]:
result.calculate_FIM(doe_object1.jac, doe_object1.design_values, jaco_involved={'FCO2':[10],'temp':[10,19]})

print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

involved flatten name: ['FCO2_index_10', 'temp_index_10', 'temp_index_19']
jaco now: {'fitted_transport_coefficient': [0.0, 0.00028439299626370484, 0.0005765679986780015, 0.0009763136290530383, 0.00140421846595018, 0.001948603966184237, 0.0024347681630843, 0.0030526560408361547, 0.0034965688371357832, 0.004146122108750205, 0.004376300040576098, 0.0050695318236648745, 0.004758926849035561, 0.005686151610007073, 0.004509954948161976, 0.011320019098093326, 0.006974129490799896, -0.012313218121562386, -0.01230862380907638, -0.011494940139300525, -0.010325523618348598, -0.009001355778082145, -0.007649775511708866, -0.006348682035195453, -0.0051430915277705935, -0.004056203067648312, -0.00309691532152101, -0.002264973943510995, -0.0015544920017873665, -0.0009563462413098023, -0.0004597904137426667, -5.351921572582796e-05, 0.0002736410067427642, 0.00053231352126204, 0.0007323238833123469, 0.0008825844020066144, 0.000991052274912363, 0.00106474581684779, 0.001109790588316173, 0.001131485504402

In [ ]:
print(np.shape(doe_object1.jac['ua']))

In [ ]:
jac = []
jac.append(doe_object1.jac['fitted_transport_coefficient'])
jac.append(doe_object1.jac['ua'])
print(np.shape(jac))

In [ ]:
jac = np.asarray(jac)

print(jac@jac.T)

In [ ]:
k_up = 212.212


k_plus = [0.000199428298248, 4.699951117693174e-05, 4.739277263415372e-05, 4.7973318219580626e-05, 4.848590670966073e-05, 4.893849938732013e-05, 4.93362877249283e-05, 4.969101036344987e-05, 5.0012457486497546e-05, 5.0306041445173156e-05, 5.0584726828579015e-05, 5.086248739998321e-05, 5.1150389267254176e-05, 5.147046013758586e-05, 5.185141920345265e-05, 5.232156281595354e-05, 5.293041438843327e-05, 5.3747524549825775e-05, 5.484493425210919e-05, 5.634472657722615e-05, 5.8419563265230855e-05, 6.124125148026447e-05, 6.511272971530605e-05, 7.04427497509817e-05, 7.760100031001833e-05, 8.730694506912328e-05, 0.00010043085700555298, 0.0001176327332325377, 0.00014049860342622408, 0.0001705909405587523, 0.0002088223361758452, 0.0002584638546666061, 0.00032165947416101516, 0.0003991706729294636, 0.0004975865575468855, 0.0006185433942340542, 0.0007620880585002776, 0.0009414833696277597, 0.0011544547221326931, 0.0014003586453413919, 0.0017053739780486918, 0.0020549752818480086, 0.0024504532519845895, 0.002941072011058184, 0.0034823523112652304, 0.00408727368420503, 0.004842120677153469, 0.005639013592289379, 0.006528512566500734, 0.007647874795368849, 0.008769336497551737, 0.01003981272211263, 0.011653396259141918, 0.01318742920543726, 0.015052678144349777, 0.017704373567782115, 0.020393267109759507, 0.022910844555995555, 0.025246940044135642, 0.027402172717012174, 0.02938107535733492, 0.031190892645138076, 0.032840600717032355, 0.03434017133572462, 0.035700038684299655, 0.036930719075571224, 0.038042544708176626, 0.039045482321676754, 0.039949014729097704, 293.15, 293.1500187845877, 293.1500423238065, 293.15007858594686, 293.15013213218157, 293.1502109412867, 293.1503098409017, 293.1504340797206, 293.1505970017368, 293.1507817763954, 293.15099688123297, 293.1512651889369, 293.1515499916254, 293.1518710200303, 293.1522660058332, 293.1526629709452, 293.15310832169513, 293.1536568090477, 293.1541752692966, 293.15476743955946, 293.1555009727011, 293.1561447086788, 293.15691817564664, 293.1578858881502, 293.15869320991015, 293.1598987095859, 293.162003952082, 293.1642267563351, 293.1650812698032, 293.1642599703442, 293.16322193267155, 293.16218805220944, 293.1611824342894, 293.16021986742675, 293.1593097721355, 293.15845746764177, 293.15766529910155, 293.1569335012573, 293.1562608405922, 293.1556450899596, 293.15508338214556, 293.15457246900195, 293.1541089094738, 293.1536892069619, 293.15330990567816, 293.15296765650754, 293.15265926207934, 293.152381704166, 293.1521321585132, 293.1519080018288, 293.15170681152364, 293.15152636085475, 293.15136461166946, 293.15121970467703, 293.15108994845184, 293.1509738081819, 293.15086989414925, 293.1507769502313, 293.15069384285664, 293.1506195504546, 293.1505531534885, 293.1504938251291, 293.1504408225886, 293.1503934791161, 293.1503511966367, 293.15031343901364, 293.1502797258982, 293.15024962713454, 293.1502227576937, 293.15, 293.1500010335017, 293.14999999841626, 293.1499999984902, 293.150000008147, 293.1500000241929, 293.1500000501105, 293.1500000914579, 293.15000015732653, 293.15000025853186, 293.15000041177353, 293.15000064370867, 293.1500009842402, 293.150001480074, 293.15000220425543, 293.15000323304025, 293.1500046907665, 293.15000676597924, 293.1500096373653, 293.15001361832213, 293.15001915941207, 293.1500266225824, 293.1500367359367, 293.15005043595204, 293.1500682670694, 293.1500917194997, 293.1501223428494, 293.150160499709, 293.15020909101906, 293.150269966785, 293.15034235013064, 293.15043194931803, 293.15053969499667, 293.1506625424434, 293.1508123121182, 293.15098583698233, 293.1511770597283, 293.15141014189754, 293.1516702383612, 293.1519494171356, 293.1522943968479, 293.1526627287503, 293.15305113226367, 293.1535434763439, 293.1540398504765, 293.1545605648515, 293.15524368162625, 293.155881737787, 293.1565635074959, 293.1574998551986, 293.1583175503289, 293.15934314197807, 293.16127538051535, 293.1635525537351, 293.1648525452506, 293.1642613219109, 293.1632543142469, 293.16223889668913, 293.1612460607743, 293.1602917353645, 293.1593863708034, 293.15853613027036, 293.1577440309363, 293.1570108374121, 293.15633573137563, 293.15571681040467, 293.15515145853936, 293.15463662027156, 293.1541690015818]



u_up = 15.4404
u_plus = [0.000199428298248, 4.6995730359055643e-05, 4.738933048588745e-05, 4.796983565167781e-05, 4.848223229920271e-05, 4.89345262440205e-05, 4.93319017990802e-05, 4.968609106246789e-05, 5.000690209932615e-05, 5.029972923544638e-05, 5.057749797412693e-05, 5.085416174106616e-05, 5.1140732717318726e-05, 5.1459135854108436e-05, 5.18379917593785e-05, 5.230546416552717e-05, 5.291082122262744e-05, 5.372335163515759e-05, 5.481480305500159e-05, 5.630663748691096e-05, 5.837095820059657e-05, 6.117898292915056e-05, 6.503221064507634e-05, 7.033849256626652e-05, 7.74665292843386e-05, 8.713269402321153e-05, 0.000100206793485153, 0.00011734748752334977, 0.0001401348622579036, 0.00017013455130856687, 0.00020825688346299146, 0.0002577620448722645, 0.0003208086441049506, 0.0003981500371588078, 0.0004963571568506762, 0.0006171070384579854, 0.0007604166840335639, 0.0009395254128473689, 0.0011522488214695081, 0.0013978545106883278, 0.0017025130629806774, 0.0020518658261506335, 0.002446982488045991, 0.0029372039997834815, 0.003478305604417322, 0.004082789856072305, 0.004837276052099247, 0.005634165068890784, 0.006523101778176079, 0.007642301857313057, 0.008764061267812382, 0.010033720560060804, 0.011647615693562255, 0.013182257485083747, 0.015045691009601935, 0.017697407813200637, 0.020388017576159753, 0.022907245839235212, 0.025244829167968646, 0.027401369798273068, 0.02938139693381415, 0.031192160613880752, 0.03284264799840861, 0.034342845122431555, 0.03570320199894807, 0.03693425117244153, 0.038046340708181955, 0.039049452366142615, 0.03995308301186225, 293.15, 293.1500184080027, 293.1500414981713, 293.15007710110245, 293.15012967265994, 293.1502071027211, 293.15030433348454, 293.1504264648424, 293.1505866979632, 293.15076851643374, 293.1509801211712, 293.1512441707967, 293.15152457749383, 293.15184049307715, 293.1522293833222, 293.15262037570017, 293.1530586939186, 293.1535989248864, 293.15410974338346, 293.15469255848456, 293.1554153383604, 293.15604973360433, 293.156810864942, 293.15776474138704, 293.1585601865422, 293.159745228639, 293.1618177609046, 293.16400949110164, 293.16485799721784, 293.1640504873682, 293.1630274635865, 293.1620084216792, 293.16101719512153, 293.16006839351536, 293.15917132338785, 293.15833123894913, 293.15755045779747, 293.15682921287254, 293.15616628664094, 293.15555948149245, 293.15500596765213, 293.1545025391385, 293.1540458004891, 293.15363230131084, 293.1532586315283, 293.15292148710705, 293.15261771368375, 293.15234433376423, 293.15209856180417, 293.15187781045677, 293.15167969047957, 293.1515020061918, 293.1513427479059, 293.1512000824006, 293.15107234223075, 293.15095801445744, 293.1508557292237, 293.150764248477, 293.1506824550494, 293.15060934223345, 293.1505440039438, 293.15048562551095, 293.15043347512807, 293.15038689595207, 293.1503452988402, 293.15030815570304, 293.1502749934372, 293.15024538840726, 293.1502189614351, 293.15, 293.15000101667835, 293.1499999967782, 293.14999999637007, 293.1500000053601, 293.15000002062084, 293.1500000455639, 293.1500000856281, 293.1500001497175, 293.1500002484333, 293.1500003981226, 293.15000062490964, 293.15000095812104, 293.1500014434955, 293.1500021526689, 293.1500031604722, 293.15000458871697, 293.15000662247235, 293.15000943722265, 293.1500133401244, 293.15001877400346, 293.15002609469735, 293.15003601620936, 293.1500494607539, 293.1500669644732, 293.15008998931745, 293.15012006640353, 293.1501575533052, 293.15020529617476, 293.1502651353667, 293.1503363013444, 293.1504243952507, 293.15053038079844, 293.1506512324167, 293.1507985607625, 293.1509693468825, 293.15115753877086, 293.1513869065054, 293.1516430139224, 293.1519178344721, 293.15225741470965, 293.1526202399741, 293.15300262247524, 293.1534873946435, 293.15397652596516, 293.1544891528913, 293.15516196817015, 293.1557909227513, 293.1564619600547, 293.15738445901644, 293.1581904910404, 293.159198857473, 293.16110029913307, 293.1633447198417, 293.1646315010669, 293.16405209130033, 293.1630597110052, 293.1620588572909, 293.1610802111473, 293.1601395089529, 293.15924707464364, 293.1584089967866, 293.15762825623193, 293.1569056089708, 293.1562402477669, 293.15563029497986, 293.1550731686202, 293.1545658529663, 293.15410509711734]

k_do = 211.788

k_minus = [0.000199428298248, 4.699261639719606e-05, 4.738709614077897e-05, 4.796810594389474e-05, 4.8480856659298244e-05, 4.893339598739968e-05, 4.933089896422115e-05, 4.968508918752209e-05, 5.0005790637777396e-05, 5.029837793234753e-05, 5.0575736464842685e-05, 5.085179837689162e-05, 5.1137521177518035e-05, 5.1454727765583866e-05, 5.183194103341992e-05, 5.229719316903412e-05, 5.289950112178491e-05, 5.370789542099725e-05, 5.479383427623903e-05, 5.627816890084082e-05, 5.833245031246168e-05, 6.112731433562986e-05, 6.496283801263728e-05, 7.024597732029174e-05, 7.734444294591548e-05, 8.697156312737334e-05, 9.999665035807227e-05, 0.00011707704159011051, 0.0001397869254949013, 0.00016969501788642951, 0.00020770943679463493, 0.0002570796034561222, 0.00031997851391728693, 0.0003971516899583889, 0.0004951518631632794, 0.0006156963675640113, 0.0007587729758784577, 0.0009375973107740886, 0.0011500744560647292, 0.0013953844354412219, 0.0016996886607171706, 0.0020487944402168383, 0.0024435532988847644, 0.0029333799679383777, 0.0034743040311163032, 0.004078356200400577, 0.004832482849470347, 0.00562936821081937, 0.006517750188467422, 0.007636786059323258, 0.008758842216472672, 0.010027696392210364, 0.011641894646538585, 0.01317714540265469, 0.01503879086088307, 0.017690524002546434, 0.020382835166440884, 0.02290369856067387, 0.025242754166601537, 0.02740058794896872, 0.029381725958094254, 0.03119342384716312, 0.032844679922729346, 0.03434549451910795, 0.035706333438603036, 0.0369377453719229, 0.038050094138219366, 0.03905337639346936, 0.03995710300761091, 293.15, 293.15001860322593, 293.15004196260975, 293.15007801332746, 293.1501312451298, 293.15020970343363, 293.1503082884652, 293.15043211049425, 293.1505946360449, 293.150779151611, 293.1509938364574, 293.15126183627285, 293.151546565722, 293.151867192537, 293.15226207519345, 293.1526592522873, 293.15310416777993, 293.15365293174295, 293.1541719924821, 293.1547635741934, 293.1554980599121, 293.15614284391023, 293.1569153058351, 293.1578849917782, 293.1586933206901, 293.15989474696784, 293.1619989154658, 293.1642276266377, 293.165096495547, 293.1642777634496, 293.16323796228846, 293.16220208183285, 293.1611944355739, 293.1602299078963, 293.15931798016254, 293.1584640045929, 293.1576703429956, 293.15693723358174, 293.156263436886, 293.1556467164397, 293.1550841919275, 293.1545725999076, 293.1541084849619, 293.15368833568704, 293.15330868177716, 293.15296616159435, 293.15265756577816, 293.152379865065, 293.15213022605275, 293.1519060169531, 293.1517048076378, 293.1515243652512, 293.1513626461255, 293.15121778614775, 293.151088090031, 293.1509720196906, 293.15086818256026, 293.15077532018137, 293.1506922970678, 293.1506180900867, 293.15055177844124, 293.1504925342993, 293.1504396140875, 293.1503923504521, 293.1503501448685, 293.1503124608775, 293.150278817913, 293.1502487856879, 293.1502219790824, 293.15, 293.15000103255085, 293.1499999966206, 293.1499999959856, 293.15000000488624, 293.15000002010265, 293.1500000450815, 293.15000008531155, 293.1500001497877, 293.1500002492269, 293.15000040013524, 293.15000062891875, 293.15000096524204, 293.15000145530547, 293.1500021715665, 293.1500031897643, 293.15000463298094, 293.1500066885694, 293.1500095342744, 293.1500134805757, 293.1500189763221, 293.15002638231294, 293.1500364206517, 293.15005002792725, 293.1500677486606, 293.15009106200233, 293.15012152804223, 293.15015951061304, 293.1502078894981, 293.15026855249346, 293.1503407137807, 293.1504300414625, 293.1505375617815, 293.15066017504444, 293.1508096423294, 293.15098299884556, 293.1511740112532, 293.15140679479543, 293.1516668745272, 293.15194588213535, 293.15229061824346, 293.1526592157564, 293.15304746752355, 293.15353973385623, 293.154036828066, 293.1545573085419, 293.15524072869977, 293.15588013640576, 293.1565613110322, 293.15749862791637, 293.15831807864345, 293.15934058924057, 293.16127027769437, 293.1635516343688, 293.1648651573508, 293.16427963750084, 293.1632710583019, 293.1622536582116, 293.1612587830522, 293.1603024657834, 293.1593952243477, 293.1585432615932, 293.1577496144452, 293.15701505359664, 293.1563387576936, 293.15571881545407, 293.155152598579, 293.1546370373982, 293.154168823115]


u_do = 15.4095

u_minus = [0.000199428298248, 4.699688828994504e-05, 4.73917060936087e-05, 4.797345405391515e-05, 4.848711795463419e-05, 4.8940714854442665e-05, 4.933938295445028e-05, 4.9694892874587365e-05, 5.00170362548322e-05, 5.0311205142843034e-05, 5.059032627030739e-05, 5.086834864595668e-05, 5.115630150086214e-05, 5.147607731501133e-05, 5.185633631850207e-05, 5.232524430538588e-05, 5.293207972740458e-05, 5.374615237552367e-05, 5.4839228147969946e-05, 5.633280507908122e-05, 5.839902237989424e-05, 6.120913044888076e-05, 6.506469132544688e-05, 7.037360969596968e-05, 7.750459513032577e-05, 8.717416080400461e-05, 0.00010025210441569612, 0.00011739708711744893, 0.00014018940893885637, 0.00017019448994122502, 0.00020832271367801717, 0.0002578347305890588, 0.0003208883197460254, 0.0003982374064635791, 0.0004964536240526351, 0.0006172117364975083, 0.0007605306928455294, 0.0009396508176501988, 0.0011523824761591665, 0.0013979991596253642, 0.0017026722252492532, 0.00205203283642823, 0.0024471662111215054, 0.0029374086888346016, 0.0034785169790837807, 0.004083029182886091, 0.004837542844775146, 0.005634433433184118, 0.006523420056227114, 0.00764265055096302, 0.0087644012157343, 0.010034153783711919, 0.011648071918406774, 0.013182696987219454, 0.01504634625499059, 0.017698176577742773, 0.02038873707107557, 0.02290790672166327, 0.025245431143470637, 0.02740191437770024, 0.029381886835303433, 0.031192599305107458, 0.03284303934639354, 0.03434319314050986, 0.035703510669287314, 0.03693452432549417, 0.038046581944136626, 0.039049664996759464, 0.039953269709426996, 293.15, 293.1500189841198, 293.1500427986093, 293.15007951790074, 293.15013373797046, 293.15021359554856, 293.15031387426853, 293.15043983449596, 293.15060508915656, 293.1507926052205, 293.1510108412605, 293.15128316284927, 293.15157235371777, 293.1518981701074, 293.15229924429923, 293.15270248733884, 293.15315454635623, 293.1537117007657, 293.15423852381826, 293.1548396140792, 293.15558503082445, 293.1562393027994, 293.15702431313434, 293.15800806468195, 293.1588284574556, 293.1600507193103, 293.16218818206727, 293.1644484673219, 293.1653233040556, 293.16449049045025, 293.16343542875376, 293.1623844681944, 293.16136219818446, 293.16038368411904, 293.15945852265423, 293.15859213295386, 293.1577869039036, 293.15704307522503, 293.1563593919305, 293.1557335867056, 293.15516274106017, 293.15464354921454, 293.15417250877033, 293.1537460613559, 293.15336069059043, 293.1530129885462, 293.15269970220845, 293.15241776117796, 293.1521642923688, 293.1519366275369, 293.15173230283386, 293.1515490536236, 293.1513848073484, 293.1512376735308, 293.15110593242883, 293.1509880236744, 293.15088253456844, 293.15078818830864, 293.15070383275093, 293.15062842966955, 293.15056104461013, 293.1505008373959, 293.15044705330666, 293.15039901493225, 293.1503561146829, 293.15031780793333, 293.1502836067661, 293.15025307428016, 293.15022581944794, 293.15, 293.1500010506729, 293.14999999993086, 293.1500000003751, 293.15000001053113, 293.150000027172, 293.1500000536391, 293.15000009572407, 293.1500001625488, 293.15000026504515, 293.1500004201006, 293.1500006546343, 293.1500009989632, 293.1500015001376, 293.1500022321965, 293.1500032722889, 293.1500047460839, 293.1500068444854, 293.15000974849755, 293.15001377496054, 293.1500193806001, 293.1500269324133, 293.1500371668178, 293.1500510348971, 293.1500690895071, 293.1500928385265, 293.15012386071896, 293.150162524923, 293.15021176671746, 293.1502734834114, 293.1503468816747, 293.1504377391525, 293.15054704725, 293.1506716873716, 293.15082363504007, 293.1509997716864, 293.1511938613345, 293.1514304194667, 293.15169454896903, 293.15197798575446, 293.15232821455464, 293.15270240353135, 293.153096779179, 293.15359675161613, 293.1541011969954, 293.15462991142596, 293.1553238182168, 293.1559724569953, 293.156664575412, 293.1576159893809, 293.1584472656362, 293.1594873993136, 293.1614485582093, 293.1637631831425, 293.16508976997807, 293.1644919835571, 293.1634685277938, 293.1624363298336, 293.1614270407744, 293.16045688820253, 293.15953651766694, 293.1586722071565, 293.15786702998173, 293.15712176519446, 293.1564355798709, 293.1558065373161, 293.15523197453945, 293.1547087813135, 293.15423360487443]

In [ ]:
print((212.212-211.788)/2/212)

In [ ]:
jac_calc = []

jac_calc_k = []
for i in range(len(k_plus)):
    jac_calc_k.append((k_plus[i]-k_minus[i])/0.002)
    
jac_calc_u = []
for i in range(len(u_plus)):
    jac_calc_u.append((u_plus[i]-u_minus[i])/0.002)

In [ ]:
jac_calc.append(jac_calc_k)
jac_calc.append(jac_calc_u)

jac_calc_array = np.asarray(jac_calc)
print(jac_calc_array@jac_calc_array.T)

In [ ]:
print(jac_calc_k)
print(jac[0])

In [ ]:
with open('20211028_heatmap0', 'rb') as f:
    output_record = pickle.load(f)
    f.close()
    
print(output_record)